<a href="https://colab.research.google.com/github/BharatXBhalla/Deep-Learning-CDAC/blob/main/Rnn/Rain_in_Austrailia_Using_Rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras import Input
from keras.layers import Dense,SimpleRNN
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import plotly
import plotly.express as ex
import plotly.graph_objects as go

In [3]:
pd.options.display.max_columns=50

In [4]:
df=pd.read_csv("/weatherAUS.csv",encoding="utf-8")

In [5]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [6]:
df.shape

(145460, 23)

In [7]:
df.isna().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [11]:
df=df[pd.isnull(df['MinTemp']) == False]
df=df[pd.isnull(df['MaxTemp']) == False]

In [13]:
df.isna().sum()

Date                 0
Location             0
MinTemp              0
MaxTemp              0
Rainfall          2249
Evaporation      61663
Sunshine         68660
WindGustDir       9243
WindGustSpeed     9182
WindDir9am        9755
WindDir3pm        3493
WindSpeed9am      1107
WindSpeed3pm      2373
Humidity9am       1449
Humidity3pm       3267
Pressure9am      13677
Pressure3pm      13671
Cloud9am         54511
Cloud3pm         57855
Temp9am            575
Temp3pm           2381
RainToday         2249
RainTomorrow      2257
dtype: int64

In [14]:
df['MedTemp']= df[['MinTemp','MaxTemp']].median(axis=1)

In [16]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,MedTemp
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No,18.15
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No,16.25
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No,19.30
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No,18.60
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,24.90


In [18]:
df['Location'].unique()

array(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'Newcastle', 'NorahHead', 'NorfolkIsland', 'Penrith', 'Richmond',
       'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown',
       'Wollongong', 'Canberra', 'Tuggeranong', 'MountGinini', 'Ballarat',
       'Bendigo', 'Sale', 'MelbourneAirport', 'Melbourne', 'Mildura',
       'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns',
       'GoldCoast', 'Townsville', 'Adelaide', 'MountGambier', 'Nuriootpa',
       'Woomera', 'Albany', 'Witchcliffe', 'PearceRAAF', 'PerthAirport',
       'Perth', 'SalmonGums', 'Walpole', 'Hobart', 'Launceston',
       'AliceSprings', 'Darwin', 'Katherine', 'Uluru'], dtype=object)

In [19]:
dfSydney = df[df["Location"] == "Sydney"]

In [29]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=dfSydney["Date"],
    y=dfSydney["MedTemp"],
    mode="lines",
    name="Median Temperatures",
    opacity=0.8,  
    line={'color':'orange','width':1}
))

fig.update_layout({'plot_bgcolor':'black'})
fig.update_xaxes(showgrid=True,gridcolor='lightgray',showline=True,zeroline=True,zerolinecolor='gray',zerolinewidth=1,
                 linewidth=1,linecolor='red',title='Date')
fig.update_yaxes(showgrid=True,gridcolor='lightgray',showline=True,zeroline=True,zerolinecolor='gray',zerolinewidth=1,
                 linewidth=1,linecolor='red',title='Degree celcuis')
fig.update_layout(title={'text':'Avg daily temp in Sydney','font':{'color':'green'}})


In [71]:
def prep_data(data,time_step):
  # create an array with in
  y_indicies = np.arange(start=time_step,stop=len(data),step=time_step)  

  y_temp = data[y_indicies]

  row_x=len(y_temp)
  print(row_x)

  x_temp= data[range(time_step * row_x)]

  x_temp = np.reshape(x_temp,(row_x,time_step,1))
  
  return x_temp,y_temp

In [73]:
X,y =  prep_data(np.array(dfSydney["MedTemp"]),7) 

476


In [74]:
X.shape

(476, 7, 1)

In [76]:
y.shape

(476,)

In [81]:
scaler = MinMaxScaler()

data_scaled = scaler.fit_transform(dfSydney[["MedTemp"]])

In [85]:
traindata,testdata = train_test_split(data_scaled,test_size=.3)

In [86]:
traindata.shape

(2337, 1)

In [87]:
testdata.shape

(1002, 1)

In [88]:
timestep = 7

X_train,y_train =prep_data(traindata,timestep)
X_test,y_test =prep_data(testdata,timestep)

333
143


In [89]:
X_train[0]

array([[0.3100616 ],
       [0.45585216],
       [0.06776181],
       [0.44353183],
       [0.41889117],
       [0.39425051],
       [0.27310062]])

In [90]:
y_train[0]

array([0.7412731])

In [166]:
model = keras.Sequential()
model.add(Input(shape=(timestep,1),name="Input_Layer"))

In [167]:
model.add(SimpleRNN(units=1024,activation="tanh"))

model.add(Dense(units=512,activation="relu"))

model.add(Dense(units=128,activation="tanh"))

model.add(Dense(units=64,activation="tanh"))

model.add(Dense(units=32,activation="tanh"))

model.add(Dense(units=1,activation="tanh"))

In [168]:
model.compile(optimizer="adam",loss="mean_squared_error",metrics=["MeanSquaredError","MeanAbsoluteError"])

In [169]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_12 (SimpleRNN)   (None, 1024)              1050624   
                                                                 
 dense_11 (Dense)            (None, 512)               524800    
                                                                 
 dense_12 (Dense)            (None, 128)               65664     
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dense_14 (Dense)            (None, 32)                2080      
                                                                 
 dense_15 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,651,457
Trainable params: 1,651,457
N

In [170]:
history= model.fit(X_train,y_train,epochs=100,batch_size=32,verbose=1,validation_data=(X_test,y_test) )

Epoch 1/100
11/11 [==============================] - 2s 134ms/step - loss: 0.3260 - mean_squared_error: 0.3260 - mean_absolute_error: 0.4880 - val_loss: 0.0940 - val_mean_squared_error: 0.0940 - val_mean_absolute_error: 0.2524
Epoch 2/100
11/11 [==============================] - 2s 145ms/step - loss: 0.0530 - mean_squared_error: 0.0530 - mean_absolute_error: 0.1866 - val_loss: 0.0385 - val_mean_squared_error: 0.0385 - val_mean_absolute_error: 0.1623
Epoch 3/100
11/11 [==============================] - 2s 153ms/step - loss: 0.0423 - mean_squared_error: 0.0423 - mean_absolute_error: 0.1703 - val_loss: 0.0422 - val_mean_squared_error: 0.0422 - val_mean_absolute_error: 0.1690
Epoch 4/100
11/11 [==============================] - 2s 152ms/step - loss: 0.0399 - mean_squared_error: 0.0399 - mean_absolute_error: 0.1619 - val_loss: 0.0337 - val_mean_squared_error: 0.0337 - val_mean_absolute_error: 0.1549
Epoch 5/100
11/11 [==============================] - 1s 111ms/step - loss: 0.0353 - mean_squ

In [171]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [172]:
history.params

{'epochs': 100, 'steps': 11, 'verbose': 1}

In [173]:
print("MSE in Train Data ",mean_squared_error(y_train,pred_train))

MSE in Train Data  0.026465690679184885


In [174]:
print("MSE in Train Data ",mean_squared_error(y_test,pred_test))

MSE in Train Data  0.03779715902642009


In [175]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=np.arange(0,len(y_test)),
    y=scaler.inverse_transform(y_test).flatten(),
    mode="lines",
    name="Actual Median Temperatures",
    opacity=0.8,  
    line={'color':'orange','width':1}
))

fig.add_trace(go.Scatter(
    x=np.arange(0,len(pred_test)),
    y=scaler.inverse_transform(pred_test).flatten(),
    mode="lines",
    name="Predicted Median Temperatures",
    opacity=0.8,  
    line={'color':'red','width':1}
))


fig.update_layout({'plot_bgcolor':'white'})
fig.update_xaxes(showgrid=True,gridcolor='lightgray',showline=True,zeroline=True,zerolinecolor='gray',zerolinewidth=1,
                 linewidth=1,linecolor='red',title='Date')
fig.update_yaxes(showgrid=True,gridcolor='lightgray',showline=True,zeroline=True,zerolinecolor='gray',zerolinewidth=1,
                 linewidth=1,linecolor='red',title='Degree celcuis')
fig.update_layout(title={'text':'Avg daily temp in Sydney','font':{'color':'green'}})


In [176]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=np.arange(0,len(y_train)),
    y=scaler.inverse_transform(y_train).flatten(),
    mode="lines",
    name="Actual Median Temperatures",
    opacity=0.8,  
    line={'color':'orange','width':1}
))

fig.add_trace(go.Scatter(
    x=np.arange(0,len(pred_train)),
    y=scaler.inverse_transform(pred_train).flatten(),
    mode="lines",
    name="Predicted Median Temperatures",
    opacity=0.8,  
    line={'color':'red','width':1}
))


fig.update_layout({'plot_bgcolor':'white'})
fig.update_xaxes(showgrid=True,gridcolor='lightgray',showline=True,zeroline=True,zerolinecolor='gray',zerolinewidth=1,
                 linewidth=1,linecolor='red',title='Date')
fig.update_yaxes(showgrid=True,gridcolor='lightgray',showline=True,zeroline=True,zerolinecolor='gray',zerolinewidth=1,
                 linewidth=1,linecolor='red',title='Degree celcuis')
fig.update_layout(title={'text':'Avg daily temp in Sydney','font':{'color':'green'}})
